# Data Analysis

In [319]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [320]:
df_experimental = pd.read_csv("DataResults/results-experimental.csv")
df_experimental = df_experimental.iloc[: , 1:]
df_experimental

,PID,Starting Interruption,Starting Task,Condition,d_age,d_gender,d_education,d_asd,d_colorblind,d_hearingimpaired,...,te_p_correctness,te_p_time,te_p_times,te_p_percentage,te_p_percentage100,te_p_resumption,te_p_resumptions,te_p_interruptions,te_p_movestotal,te_p_moveinterruptedtime
0,105602-204235,2,1,2,36,1,None,False,False,False,...,10.0,11.696433,"[0.625, 0.40625, 0.734375, 0.921875, 1.0, 1.0,...",0.62500,0.3750,13.117787,"[25.24606, 8.702699, 9.789602, 20.26899, 16.49...",NaN,uncollected,9.573046
1,110152-525169,2,1,1,21,1,College,False,False,False,...,6.5,4.975075,"[0.625, 0.40625, 0.734375, 0.921875, 1.0, 1.0,...",0.40625,0.2500,4.313637,"[3.896301, 4.78, 3.766599, 5.841499, 3.8199, 4...",NaN,uncollected,2.044195
2,112938-598944,2,2,2,23,1,University,False,False,False,...,1.0,460.337500,28.77109377,1.00000,1.0000,1.005104,"[7.921699, 0.8333001, 0.6661999, 0.6331001, 0....",458.0,28.625,1.028230
3,112992-130081,2,2,2,55,1,University,False,False,False,...,1.0,701.689220,43.85557626,1.00000,1.0000,3.104820,"[10.79079, 4.078399, 1.1658, 1.4478, 1.1486, 1...",226.0,14.125,3.131094
4,115534-782584,1,2,2,23,0,University,False,False,False,...,1.0,393.653401,24.60333756,1.00000,1.0000,2.008436,"[4.1478, 1.4955, 1.0902, 1.4623, 1.3611, 1.194...",196.0,12.25,2.257588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,985214-920725,1,1,1,22,0,University,False,False,False,...,15.5,8.426413,"[0.625, 0.40625, 0.734375, 0.921875, 1.0, 1.0,...",0.96875,0.9375,8.678775,"[10.485, 12.8889, 11.9101, 5.727599, 6.122199,...",NaN,uncollected,1.994750
123,985353-521999,1,2,2,22,1,College,False,False,False,...,1.0,278.168375,17.38552344,1.00000,1.0000,1.441287,"[2.605, 0.9156001, 0.9671, 0.8825002, 1.1008, ...",193.0,12.0625,1.498002
124,991009-286161,1,2,2,22,1,University,False,False,False,...,1.0,255.451968,15.96574801,1.00000,1.0000,1.264614,"[3.473998, 1.083, 0.867, 1.335, 0.751, 0.83400...",202.0,12.625,1.308853
125,991710-413328,2,2,2,45,0,University,False,False,False,...,1.0,277.406090,17.33788061,1.00000,1.0000,1.401041,"[4.350599, 0.9829001, 1.432201, 0.6998002, 0.8...",198.0,12.375,1.637923


## Format Variables

In [156]:
# df_experimental.replace({'N/A':1}) # hanoi accuracy and completeness is always 100%
# gender --> Male = 0, Female = 1, else = -1

df_demographics = df_experimental[[
    'd_age', 'd_gender'
]]

df_data = df_experimental[[
    'a_p_count', 'tr_p_count', 'te_p_count',
    'a_p_percentage', 'tr_p_percentage', 'te_p_percentage',
    'a_p_percentage100', 'tr_p_percentage100', 'te_p_percentage100',
    'a_p_time', 'tr_p_time', 'te_p_time',
    'a_p_resumption', 'tr_p_resumption', 'te_p_resumption',
    'a_p_moveinterruptedtime', 'tr_p_moveinterruptedtime', 'te_p_moveinterruptedtime',
    'a_i_count', 'tr_i_count', 'te_i_count',
    'a_i_percentage', 'tr_i_percentage', 'te_i_percentage',
    'a_i_time', 'tr_i_time', 'te_i_time'
]]

df_survey = df_experimental[[
    'a_p_e_effort', 'te_p_e_effort',
    'a_p_e_confidence', 'te_p_e_confidence',
    'a_i_e_effort', 'te_i_e_effort',
    'a_i_e_confidence', 'te_i_e_confidence'
]]

x_task_dummies = pd.get_dummies(df_experimental["Starting Task"], prefix='task', drop_first=False)
x_interruption_dummies = pd.get_dummies(df_experimental["Starting Interruption"], prefix='interruption', drop_first=False)
x_condition_dummies = pd.get_dummies(df_experimental["Condition"], prefix='condition', drop_first=False)

df_x = pd.concat((
    df_demographics,            # does any of the demographics account for improvement in resumption lag?
    df_data,                    # does any of the performance metrics account for improvement in resumption lag?
    df_survey,                  # does any of the cognitive load metrics account for improvement in resumption lag?
    x_task_dummies,             # does the starting task account for improvement in resumption lag?
    x_interruption_dummies,     # does the starting interruption account for improvement in resumption lag?
    x_condition_dummies         # does the condition participants are placed in account 
), axis=1)

df_x

,d_age,d_gender,a_p_count,tr_p_count,te_p_count,a_p_percentage,tr_p_percentage,te_p_percentage,a_p_percentage100,tr_p_percentage100,...,a_i_e_effort,te_i_e_effort,a_i_e_confidence,te_i_e_confidence,task_1,task_2,interruption_1,interruption_2,condition_1,condition_2
0,36,1,16,16,16,0.90625,0.765625,0.62500,0.8750,0.6875,...,4,4,4,6,1,0,0,1,0,1
1,21,1,16,16,16,0.68750,1.000000,0.40625,0.5625,1.0000,...,6,4,7,5,1,0,0,1,1,0
2,23,1,16,16,16,1.00000,1.000000,1.00000,1.0000,1.0000,...,6,4,4,6,0,1,0,1,0,1
3,55,1,16,16,16,1.00000,1.000000,1.00000,1.0000,1.0000,...,6,6,7,7,0,1,0,1,0,1
4,23,0,16,16,16,1.00000,1.000000,1.00000,1.0000,1.0000,...,6,6,4,5,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,22,0,16,16,16,0.93750,1.000000,0.96875,0.9375,1.0000,...,4,5,7,7,1,0,1,0,1,0
123,22,1,16,16,16,1.00000,1.000000,1.00000,1.0000,1.0000,...,4,4,6,7,0,1,1,0,0,1
124,22,1,16,16,16,1.00000,1.000000,1.00000,1.0000,1.0000,...,6,6,7,6,0,1,1,0,0,1
125,45,0,16,16,16,1.00000,1.000000,1.00000,1.0000,1.0000,...,6,6,5,6,0,1,0,1,0,1


In [233]:
# helpers for interpreting t-test statistics

def is_improvement(change):
    if (change > 0):
        return "increased [+]"
    return "decreased [-]"

def is_significant(p):
    if (p <= 0.005):
        return "extremely significantly [**]"
    elif (p <= 0.05):
        return "significantly [*]"
    elif (p <= 0.03):
        return "slightly"
    return ""

def compute_improvement(col1, col2):
    col3 = col1 - col2
    avg = sum(col3)/len(col3)
    t, p = stats.ttest_ind(col1, col2)
    return t, p, avg

## Resumption Lag

In [234]:
from scipy import stats
import statsmodels.api as sm

#### Improvement in resumption lag (t-test)

In [235]:
# resumption lag
t_resumption, p_resumption, diff_resumption = compute_improvement(
    df_experimental['a_p_resumption'], 
    df_experimental['te_p_resumption'])
print('\nRESUMPTION (all): t={t}, p={p}, d={d}'.format(t=t_resumption, p=p_resumption, d=diff_resumption))
print('Participants {significance} {change} their resumption lag regardless of the task.'.format(
    change=is_improvement(diff_resumption), significance=is_significant(p_resumption)))

# resumption lag in hanoi
t_hanoi_resumption, p_hanoi_resumption, diff_hanoi_resumption = compute_improvement(
    df_experimental[df_experimental['a_p_name'] == 'hanoi']['a_p_resumption'], 
    df_experimental[df_experimental['te_p_name'] == 'hanoi']['te_p_resumption'])
print('\nRESUMPTION (hanoi): t={t}, p={p}, d={d}'.format(t=t_hanoi_resumption, p=p_hanoi_resumption, d=diff_hanoi_resumption))
print('Participants {significance} {change} their resumption lag on the hanoi task.'.format(
    change=is_improvement(diff_hanoi_resumption), significance=is_significant(p_hanoi_resumption)))

# resumption lag in draw
t_draw_resumption, p_draw_resumption, diff_draw_resumption = compute_improvement(
    df_experimental[df_experimental['a_p_name'] == 'draw']['a_p_resumption'], 
    df_experimental[df_experimental['te_p_name'] == 'draw']['te_p_resumption'])
print('\nRESUMPTION (draw): t={t}, p={p}, d={d}'.format(t=t_draw_resumption, p=p_draw_resumption, d=diff_draw_resumption))
print('Participants {significance} {change} their resumption lag on the draw task.'.format(
    change=is_improvement(diff_draw_resumption), significance=is_significant(p_draw_resumption)))


RESUMPTION (all): t=4.048446392712252, p=6.865163191814581e-05, d=2.103470048905512
Participants extremely significantly [**] increased [+] their resumption lag regardless of the task.

RESUMPTION (hanoi): t=10.865998962818843, p=5.764994517649682e-20, d=2.306085631333334
Participants extremely significantly [**] increased [+] their resumption lag on the hanoi task.

RESUMPTION (draw): t=3.475074227446791, p=0.0007115008769777596, d=1.8842466318524589
Participants extremely significantly [**] increased [+] their resumption lag on the draw task.


#### Predictors of the improvement in resumption lag (ANOVA)

In [256]:
resumption_lag_y = df_resumption_change
resumption_lag_x = df_x.drop(columns=['te_p_resumption', 'a_p_resumption'])

In [161]:
model = sm.OLS(resumption_lag_y, resumption_lag_x)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.743
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     8.130
Date:                Sat, 11 Dec 2021   Prob (F-statistic):           7.07e-16
Time:                        13:09:31   Log-Likelihood:                -185.47
No. Observations:                 127   AIC:                             438.9
Df Residuals:                      93   BIC:                             535.6
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
d_age                   

## Interruption Lag

#### Improvement in interruption lag (t-test)

In [237]:
# interruption lag
t_interruption, p_interruption, diff_interruption = compute_improvement(
    df_experimental['a_p_moveinterruptedtime'], 
    df_experimental['te_p_moveinterruptedtime'])
print('\nINTERRUPTION (all): t={t}, p={p}, d={d}'.format(t=t_interruption, p=p_interruption, d=diff_interruption))
print('Participants {significance} {change} their interruption lag regardless of the task.'.format(
    change=is_improvement(diff_interruption), significance=is_significant(p_interruption)))

# interruption lag in hanoi
t_hanoi_interruption, p_hanoi_interruption, diff_hanoi_interruption = compute_improvement(
    df_experimental[df_experimental['a_p_name'] == 'hanoi']['a_p_moveinterruptedtime'], 
    df_experimental[df_experimental['te_p_name'] == 'hanoi']['te_p_moveinterruptedtime'])
print('\nINTERRUPTION (hanoi): t={t}, p={p}, d={d}'.format(t=t_hanoi_interruption, p=p_hanoi_interruption, d=diff_hanoi_interruption))
print('Participants {significance} {change} their interruption lag on the hanoi task.'.format(
    change=is_improvement(diff_hanoi_interruption), significance=is_significant(p_hanoi_interruption)))

# interruption lag in draw
t_draw_interruption, p_draw_interruption, diff_draw_interruption = compute_improvement(
    df_experimental[df_experimental['a_p_name'] == 'draw']['a_p_moveinterruptedtime'], 
    df_experimental[df_experimental['te_p_name'] == 'draw']['te_p_moveinterruptedtime'])
print('\nINTERRUPTION (draw): t={t}, p={p}, d={d}'.format(t=t_draw_interruption, p=p_draw_interruption, d=diff_draw_interruption))
print('Participants {significance} {change} their interruption lag on the draw task.'.format(
    change=is_improvement(diff_draw_interruption), significance=is_significant(p_draw_interruption)))


INTERRUPTION (all): t=2.849107497174644, p=0.004746388098772658, d=1.6394336355118109
Participants extremely significantly [**] increased [+] their interruption lag regardless of the task.

INTERRUPTION (hanoi): t=-2.522616679043625, p=0.01285473224363405, d=-1.8669214495909097
Participants significantly [*] decreased [-] their interruption lag on the hanoi task.

INTERRUPTION (draw): t=8.471229401869412, p=7.149461351088171e-14, d=5.433194875131148
Participants extremely significantly [**] increased [+] their interruption lag on the draw task.


#### Predictors of the improvement in interruption lag (ANOVA)

In [144]:
interruption_lag_y = df_interruption_change
interruption_lag_x = df_x.drop(columns=['te_p_moveinterruptedtime', 'a_p_moveinterruptedtime'])

In [145]:
model = sm.OLS(interruption_lag_y, interruption_lag_x)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.548
Model:                            OLS   Adj. R-squared:                  0.419
Method:                 Least Squares   F-statistic:                     4.248
Date:                Sat, 11 Dec 2021   Prob (F-statistic):           5.21e-08
Time:                        10:10:16   Log-Likelihood:                -362.47
No. Observations:                 127   AIC:                             782.9
Df Residuals:                      98   BIC:                             865.4
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
d_age                -0.0443      0.06

## Accuracy in completing primary tasks
    - for draw task as measured by percentage of tasks solved correctly
    - for hanoi task as measured by moves taken to solve

#### During draw task

In [285]:
# percentage of correct memory tasks
t_draw_correctness, p_draw_correctness, diff_draw_correctness = compute_improvement(
    df_experimental[df_experimental['a_p_name'] == 'draw']['a_p_correctness'], 
    df_experimental[df_experimental['te_p_name'] == 'draw']['te_p_correctness'])
print('\nCORRECTNESS (draw): t={t}, p={p}, d={d}'.format(t=t_draw_correctness, p=p_draw_correctness, d=diff_draw_correctness))
print('Participants {significance} {change} in how many draw tasks they answered correctly.'.format(
    change=is_improvement(diff_draw_correctness), significance=is_significant(p_draw_correctness)))

# correctness of math interruptions during draw
t_draw_math_correctness, p_draw_math_correctness, diff_draw_math_correctness = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['a_i_name'] == 'math')]['a_p_correctness'], 
    df_experimental[(df_experimental['te_p_name'] == 'draw') & (df_experimental['te_i_name'] == 'math')]['te_p_correctness'])
print('\nCORRECTNESS (draw-math): t={t}, p={p}, d={d}'.format(t=t_draw_math_correctness, p=p_draw_math_correctness, d=diff_draw_math_correctness))
print('Participants {significance} {change} in the correctness to solve a draw task when experiencing math interruptions.'.format(
    change=is_improvement(diff_draw_math_correctness), significance=is_significant(p_draw_math_correctness)))

# correctness of stroop interruptions during draw
t_draw_stroop_correctness, p_draw_stroop_correctness, diff_draw_stroop_correctness = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['a_i_name'] == 'stroop')]['a_p_correctness'], 
    df_experimental[(df_experimental['te_p_name'] == 'draw') & (df_experimental['te_i_name'] == 'stroop')]['te_p_correctness'])
print('\nCORRECTNESS (draw-stroop): t={t}, p={p}, d={d}'.format(t=t_draw_stroop_correctness, p=p_draw_stroop_correctness, d=diff_draw_stroop_correctness))
print('Participants {significance} {change} in the correctness to solve a draw task when experiencing stroop.'.format(
    change=is_improvement(diff_draw_stroop_correctness), significance=is_significant(p_draw_stroop_correctness)))


CORRECTNESS (draw): t=-2.393450012800997, p=0.01824063068827633, d=-1.0163934426229508
Participants significantly [*] decreased [-] in how many draw tasks they answered correctly.

CORRECTNESS (draw-math): t=-1.4848870525703297, p=0.14280892745096355, d=-0.9354838709677419
Participants  decreased [-] in the correctness to solve a draw task when experiencing math interruptions.

CORRECTNESS (draw-stroop): t=-1.9141043215283617, p=0.060544996624760275, d=-1.1
Participants  decreased [-] in the correctness to solve a draw task when experiencing stroop.


#### During hanoi task

In [284]:
# number of hanoi moves
t_hanoi_moves, p_hanoi_moves, diff_hanoi_moves = compute_improvement(
    df_experimental[df_experimental['a_p_name'] == 'hanoi']['a_p_movestotal'].astype(float), 
    df_experimental[df_experimental['te_p_name'] == 'hanoi']['te_p_movestotal'].astype(float))
print('\nMOVES TAKEN (hanoi): t={t}, p={p}, d={d}'.format(t=t_hanoi_moves, p=p_hanoi_moves, d=diff_hanoi_moves))
print('Participants {significance} {change} in the number of moves needed to complete a hanoi task.'.format(
    change=is_improvement(diff_hanoi_moves), significance=is_significant(p_hanoi_moves)))

# moves of math interruptions during hanoi
t_hanoi_math_moves, p_hanoi_math_moves, diff_hanoi_math_moves = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['a_i_name'] == 'math')]['a_p_movestotal'].astype(float), 
    df_experimental[(df_experimental['te_p_name'] == 'hanoi') & (df_experimental['te_i_name'] == 'math')]['te_p_movestotal'].astype(float))
print('\nMOVES TAKEN (hanoi-math): t={t}, p={p}, d={d}'.format(t=t_hanoi_math_moves, p=p_hanoi_math_moves, d=diff_hanoi_math_moves))
print('Participants {significance} {change} in the moves to solve a hanoi task when experiencing math interruptions.'.format(
    change=is_improvement(diff_hanoi_math_moves), significance=is_significant(p_hanoi_math_moves)))

# moves of stroop interruptions during hanoi
t_hanoi_stroop_moves, p_hanoi_stroop_moves, diff_hanoi_stroop_moves = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['a_i_name'] == 'stroop')]['a_p_movestotal'].astype(float), 
    df_experimental[(df_experimental['te_p_name'] == 'hanoi') & (df_experimental['te_i_name'] == 'stroop')]['te_p_movestotal'].astype(float))
print('\nMOVES TAKEN (hanoi-stroop): t={t}, p={p}, d={d}'.format(t=t_hanoi_stroop_moves, p=p_hanoi_stroop_moves, d=diff_hanoi_stroop_moves))
print('Participants {significance} {change} in the moves to solve a hanoi task when experiencing stroop.'.format(
    change=is_improvement(diff_hanoi_stroop_moves), significance=is_significant(p_hanoi_stroop_moves)))


MOVES TAKEN (hanoi): t=6.2014457283443205, p=6.898642311548927e-09, d=5.517045454545454
Participants extremely significantly [**] increased [+] in the number of moves needed to complete a hanoi task.

MOVES TAKEN (hanoi-math): t=3.8643042740838847, p=0.0002836013451029663, d=5.739583333333333
Participants extremely significantly [**] increased [+] in the moves to solve a hanoi task when experiencing math interruptions.

MOVES TAKEN (hanoi-stroop): t=5.158722310358425, p=2.2153135773664525e-06, d=5.331597222222222
Participants extremely significantly [**] increased [+] in the moves to solve a hanoi task when experiencing stroop.


## Speed in completing primary tasks
    - as measured in time (seconds) taken to solve

In [264]:
# time spent on primary tasks
t_time, p_time, diff_time = compute_improvement(
    df_experimental['a_p_time'], 
    df_experimental['te_p_time'])
print('\nTIME SPENT (any-any): t={t}, p={p}, d={d}'.format(t=t_time, p=p_time, d=diff_time))
print('Participants {significance} {change} in the time to solve any primary task'.format(
    change=is_improvement(diff_time), significance=is_significant(p_time)))


TIME SPENT (any-any): t=-0.7625881186910098, p=0.4464222873140279, d=-17.982087475826766
Participants  decreased [-] in the time to solve any primary task


#### During draw task

In [283]:
# time spent on draw task
t_draw_time, p_draw_time, diff_draw_time = compute_improvement(
    df_experimental[df_experimental['a_p_name'] == 'draw']['a_p_time'], 
    df_experimental[df_experimental['te_p_name'] == 'draw']['te_p_time'])
print('\nTIME SPENT (draw-any): t={t}, p={p}, d={d}'.format(t=t_draw_time, p=p_draw_time, d=diff_draw_time))
print('Participants {significance} {change} in the time to solve a draw task.'.format(
    change=is_improvement(diff_draw_time), significance=is_significant(p_draw_time)))

# time of math interruptions during draw
t_draw_math_time, p_draw_math_time, diff_draw_math_time = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['a_i_name'] == 'math')]['a_p_time'], 
    df_experimental[(df_experimental['te_p_name'] == 'draw') & (df_experimental['te_i_name'] == 'math')]['te_p_time'])
print('\nTIME SPENT (draw-math): t={t}, p={p}, d={d}'.format(t=t_draw_math_time, p=p_draw_math_time, d=diff_draw_math_time))
print('Participants {significance} {change} in the time to solve a draw task when experiencing math interruptions.'.format(
    change=is_improvement(diff_draw_math_time), significance=is_significant(p_draw_math_time)))

# time of stroop interruptions during draw
t_draw_stroop_time, p_draw_stroop_time, diff_draw_stroop_time = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['a_i_name'] == 'stroop')]['a_p_time'], 
    df_experimental[(df_experimental['te_p_name'] == 'draw') & (df_experimental['te_i_name'] == 'stroop')]['te_p_time'])
print('\nTIME SPENT (draw-stroop): t={t}, p={p}, d={d}'.format(t=t_draw_stroop_time, p=p_draw_stroop_time, d=diff_draw_stroop_time))
print('Participants {significance} {change} in the time to solve a draw task when experiencing stroop.'.format(
    change=is_improvement(diff_draw_stroop_time), significance=is_significant(p_draw_stroop_time)))


TIME SPENT (draw-any): t=2.0995177261376186, p=0.03786670590602681, d=1.0058557404918038
Participants significantly [*] increased [+] in the time to solve a draw task.

TIME SPENT (draw-math): t=1.241436167801279, p=0.21927611915490386, d=0.7212125481290326
Participants  increased [+] in the time to solve a draw task when experiencing math interruptions.

TIME SPENT (draw-stroop): t=1.7386290109383455, p=0.08740609582280973, d=1.2999870392666666
Participants  increased [+] in the time to solve a draw task when experiencing stroop.


#### During hanoi task

In [282]:
# time spent on hanoi task
t_hanoi_time, p_hanoi_time, diff_hanoi_time = compute_improvement(
    df_experimental[df_experimental['a_p_name'] == 'hanoi']['a_p_time'], 
    df_experimental[df_experimental['te_p_name'] == 'hanoi']['te_p_time'])
print('\nTIME SPENT (hanoi-any): t={t}, p={p}, d={d}'.format(t=t_hanoi_time, p=p_hanoi_time, d=diff_hanoi_time))
print('Participants {significance} {change} in the time to solve a hanoi task.'.format(
    change=is_improvement(diff_hanoi_time), significance=is_significant(p_hanoi_time)))

# time of math interruptions during hanoi
t_hanoi_math_time, p_hanoi_math_time, diff_hanoi_math_time = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['a_i_name'] == 'math')]['a_p_time'], 
    df_experimental[(df_experimental['te_p_name'] == 'hanoi') & (df_experimental['te_i_name'] == 'math')]['te_p_time'])
print('\nTIME SPENT (hanoi-math): t={t}, p={p}, d={d}'.format(t=t_hanoi_math_time, p=p_hanoi_math_time, d=diff_hanoi_math_time))
print('Participants {significance} {change} in the time to solve a hanoi task when experiencing math interruptions.'.format(
    change=is_improvement(diff_hanoi_math_time), significance=is_significant(p_hanoi_math_time)))

# time of stroop interruptions during hanoi
t_hanoi_stroop_time, p_hanoi_stroop_time, diff_hanoi_stroop_time = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['a_i_name'] == 'stroop')]['a_p_time'], 
    df_experimental[(df_experimental['te_p_name'] == 'hanoi') & (df_experimental['te_i_name'] == 'stroop')]['te_p_time'])
print('\nTIME SPENT (hanoi-stroop): t={t}, p={p}, d={d}'.format(t=t_hanoi_stroop_time, p=p_hanoi_stroop_time, d=diff_hanoi_stroop_time))
print('Participants {significance} {change} in the time to solve a hanoi task when experiencing stroop.'.format(
    change=is_improvement(diff_hanoi_stroop_time), significance=is_significant(p_hanoi_stroop_time)))


TIME SPENT (hanoi-any): t=-1.5622677908977638, p=0.12065602872504214, d=-35.531550145454545
Participants  decreased [-] in the time to solve a hanoi task.

TIME SPENT (hanoi-math): t=-0.7131182925191635, p=0.47863205014343113, d=-30.188741210000003
Participants  decreased [-] in the time to solve a hanoi task when experiencing math interruptions.

TIME SPENT (hanoi-stroop): t=-1.9315167456965248, p=0.05746838951937206, d=-39.983890925000004
Participants  decreased [-] in the time to solve a hanoi task when experiencing stroop.


## Accuracy in completing interruptions
    - as measured by the percentage of interrupting tasks correctly answered

In [257]:
# accuracy completing interruptions
t_accuracy, p_accuracy, diff_accuracy = compute_improvement(
    df_experimental['a_i_percentage'], 
    df_experimental['te_i_percentage'])
print('\nACCURACY OF INTERRUPTIONS (any-any): t={t}, p={p}, d={d}'.format(t=t_accuracy, p=p_accuracy, d=diff_accuracy))
print('Participants {significance} {change} in the correctness of their interruption responses regardless of task.'.format(
    change=is_improvement(diff_accuracy), significance=is_significant(p_accuracy)))


ACCURACY OF INTERRUPTIONS (any-any): t=0.5577192076550433, p=0.577531234213471, d=0.00826612679527558
Participants  increased [+] in the correctness of their interruption responses regardless of task.


#### During draw task

In [281]:
# accuracy of interruptions during draw
t_draw_accuracy, p_draw_accuracy, diff_draw_accuracy = compute_improvement(
    df_experimental[df_experimental['a_p_name'] == 'draw']['a_i_percentage'], 
    df_experimental[df_experimental['te_p_name'] == 'draw']['te_i_percentage'])
print('\nACCURACY OF INTERRUPTIONS (draw-any): t={t}, p={p}, d={d}'.format(t=t_draw_accuracy, p=p_draw_accuracy, d=diff_draw_accuracy))
print('Participants {significance} {change} in the number of interruptions completed during the draw task.'.format(
    change=is_improvement(diff_draw_accuracy), significance=is_significant(p_draw_accuracy)))

# accuracy of math interruptions during draw
t_draw_math_accuracy, p_draw_math_accuracy, diff_draw_math_accuracy = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['a_i_name'] == 'math')]['a_i_percentage'], 
    df_experimental[(df_experimental['te_p_name'] == 'draw') & (df_experimental['te_i_name'] == 'math')]['te_i_percentage'])
print('\nACCURACY OF INTERRUPTIONS (draw-math): t={t}, p={p}, d={d}'.format(t=t_draw_math_accuracy, p=p_draw_math_accuracy, d=diff_draw_math_accuracy))
print('Participants {significance} {change} in the number of math interruptions completed during the draw task.'.format(
    change=is_improvement(diff_draw_math_accuracy), significance=is_significant(p_draw_math_accuracy)))

# accuracy of stroop interruptions during draw
t_draw_stroop_accuracy, p_draw_stroop_accuracy, diff_draw_stroop_accuracy = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['a_i_name'] == 'stroop')]['a_i_percentage'], 
    df_experimental[(df_experimental['te_p_name'] == 'draw') & (df_experimental['te_i_name'] == 'stroop')]['te_i_percentage'])
print('\nACCURACY OF INTERRUPTIONS (draw-stroop): t={t}, p={p}, d={d}'.format(t=t_draw_stroop_accuracy, p=p_draw_stroop_accuracy, d=diff_draw_stroop_accuracy))
print('Participants {significance} {change} in the number of stroop interruptions completed during the draw task.'.format(
    change=is_improvement(diff_draw_stroop_accuracy), significance=is_significant(p_draw_stroop_accuracy)))


ACCURACY OF INTERRUPTIONS (draw-any): t=0.033685461574074496, p=0.9731839390867589, d=0.000773746049180326
Participants  increased [+] in the number of interruptions completed during the draw task.

ACCURACY OF INTERRUPTIONS (draw-math): t=0.12865100145638536, p=0.8980640796827879, d=0.004320622548387108
Participants  increased [+] in the number of math interruptions completed during the draw task.

ACCURACY OF INTERRUPTIONS (draw-stroop): t=-0.11627146480561182, p=0.9078390574940828, d=-0.002891359666666682
Participants  decreased [-] in the number of stroop interruptions completed during the draw task.


#### During hanoi task

In [280]:
# accuracy of interruptions during hanoi
t_hanoi_accuracy, p_hanoi_accuracy, diff_hanoi_accuracy = compute_improvement(
    df_experimental[df_experimental['a_p_name'] == 'hanoi']['a_i_percentage'], 
    df_experimental[df_experimental['te_p_name'] == 'hanoi']['te_i_percentage'])
print('\nACCURACY OF INTERRUPTIONS (hanoi-any): t={t}, p={p}, d={d}'.format(t=t_hanoi_accuracy, p=p_hanoi_accuracy, d=diff_hanoi_accuracy))
print('Participants {significance} {change} in the number of interruptions completed during the hanoi task.'.format(
    change=is_improvement(diff_hanoi_accuracy), significance=is_significant(p_hanoi_accuracy)))

# accuracy of math interruptions during hanoi
t_hanoi_math_accuracy, p_hanoi_math_accuracy, diff_hanoi_math_accuracy = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['a_i_name'] == 'math')]['a_i_percentage'], 
    df_experimental[(df_experimental['te_p_name'] == 'hanoi') & (df_experimental['te_i_name'] == 'math')]['te_i_percentage'])
print('\nACCURACY OF INTERRUPTIONS (hanoi-math): t={t}, p={p}, d={d}'.format(t=t_hanoi_math_accuracy, p=p_hanoi_math_accuracy, d=diff_hanoi_math_accuracy))
print('Participants {significance} {change} in the number of math interruptions completed during the hanoi task.'.format(
    change=is_improvement(diff_hanoi_math_accuracy), significance=is_significant(p_hanoi_math_accuracy)))

# accuracy of stroop interruptions during hanoi
t_hanoi_stroop_accuracy, p_hanoi_stroop_accuracy, diff_hanoi_stroop_accuracy = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['a_i_name'] == 'stroop')]['a_i_percentage'], 
    df_experimental[(df_experimental['te_p_name'] == 'hanoi') & (df_experimental['te_i_name'] == 'stroop')]['te_i_percentage'])
print('\nACCURACY OF INTERRUPTIONS (hanoi-stroop): t={t}, p={p}, d={d}'.format(t=t_hanoi_stroop_accuracy, p=p_hanoi_stroop_accuracy, d=diff_hanoi_stroop_accuracy))
print('Participants {significance} {change} in the number of stroop interruptions completed during the hanoi task.'.format(
    change=is_improvement(diff_hanoi_stroop_accuracy), significance=is_significant(p_hanoi_stroop_accuracy)))


ACCURACY OF INTERRUPTIONS (hanoi-any): t=0.7929583899871729, p=0.4292468781991946, d=0.015190902939393925
Participants  increased [+] in the number of interruptions completed during the hanoi task.

ACCURACY OF INTERRUPTIONS (hanoi-math): t=1.6952836617594977, p=0.09538437938764352, d=0.048804240366666654
Participants  increased [+] in the number of math interruptions completed during the hanoi task.

ACCURACY OF INTERRUPTIONS (hanoi-stroop): t=-0.6543448112513663, p=0.5150338629013747, d=-0.012820211583333345
Participants  decreased [-] in the number of stroop interruptions completed during the hanoi task.


## Speed in completing interruptions
    - as measured by the number of interrupting tasks completed within the 10s interruption period

In [252]:
# number of interruptions
t_interrupts, p_interrupts, diff_interrupts = compute_improvement(
    df_experimental['a_i_count'], 
    df_experimental['te_i_count'])
print('\nINTERRUPTIONS TOTAL (any-any): t={t}, p={p}, d={d}'.format(t=t_interrupts, p=p_interrupts, d=diff_interrupts))
print('Participants {significance} {change} in the number of interruptions completed regardless of task.'.format(
    change=is_improvement(diff_interrupts), significance=is_significant(p_interrupts)))


INTERRUPTIONS TOTAL (any-any): t=-4.0105978079467395, p=7.985779159586052e-05, d=-7.9921259842519685
Participants extremely significantly [**] decreased [-] in the number of interruptions completed regardless of task.


#### During draw task

In [279]:
# number of interruptions during draw
t_draw_interrupts, p_draw_interrupts, diff_draw_interrupts = compute_improvement(
    df_experimental[df_experimental['a_p_name'] == 'draw']['a_i_count'], 
    df_experimental[df_experimental['te_p_name'] == 'draw']['te_i_count'])
print('\nINTERRUPTIONS TOTAL (draw-any): t={t}, p={p}, d={d}'.format(t=t_draw_interrupts, p=p_draw_interrupts, d=diff_draw_interrupts))
print('Participants {significance} {change} in the number of interruptions completed during the draw task.'.format(
    change=is_improvement(diff_draw_interrupts), significance=is_significant(p_draw_interrupts)))

# number of math interruptions during draw
t_draw_math_interrupts, p_draw_math_interrupts, diff_draw_math_interrupts = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['a_i_name'] == 'math')]['a_i_count'], 
    df_experimental[(df_experimental['te_p_name'] == 'draw') & (df_experimental['te_i_name'] == 'math')]['te_i_count'])
print('\nINTERRUPTIONS TOTAL (draw-math): t={t}, p={p}, d={d}'.format(t=t_draw_math_interrupts, p=p_draw_math_interrupts, d=diff_draw_math_interrupts))
print('Participants {significance} {change} in the number of math interruptions completed during the draw task.'.format(
    change=is_improvement(diff_draw_math_interrupts), significance=is_significant(p_draw_math_interrupts)))

# number of stroop interruptions during draw
t_draw_stroop_interrupts, p_draw_stroop_interrupts, diff_draw_stroop_interrupts = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['a_i_name'] == 'stroop')]['a_i_count'], 
    df_experimental[(df_experimental['te_p_name'] == 'draw') & (df_experimental['te_i_name'] == 'stroop')]['te_i_count'])
print('\nINTERRUPTIONS TOTAL (draw-stroop): t={t}, p={p}, d={d}'.format(t=t_draw_stroop_interrupts, p=p_draw_stroop_interrupts, d=diff_draw_stroop_interrupts))
print('Participants {significance} {change} in the number of stroop interruptions completed during the draw task.'.format(
    change=is_improvement(diff_draw_stroop_interrupts), significance=is_significant(p_draw_stroop_interrupts)))


INTERRUPTIONS TOTAL (draw-any): t=-3.495630900182389, p=0.000663591247395759, d=-9.442622950819672
Participants extremely significantly [**] decreased [-] in the number of interruptions completed during the draw task.

INTERRUPTIONS TOTAL (draw-math): t=-2.7557039305778943, p=0.007744392499144166, d=-10.709677419354838
Participants significantly [*] decreased [-] in the number of math interruptions completed during the draw task.

INTERRUPTIONS TOTAL (draw-stroop): t=-2.623818063227064, p=0.011093804477621472, d=-8.133333333333333
Participants significantly [*] decreased [-] in the number of stroop interruptions completed during the draw task.


#### During hanoi task

In [278]:
# number of interruptions during hanoi
t_hanoi_interrupts, p_hanoi_interrupts, diff_hanoi_interrupts = compute_improvement(
    df_experimental[df_experimental['a_p_name'] == 'hanoi']['a_i_count'], 
    df_experimental[df_experimental['te_p_name'] == 'hanoi']['te_i_count'])
print('\nINTERRUPTIONS TOTAL (hanoi-any): t={t}, p={p}, d={d}'.format(t=t_hanoi_interrupts, p=p_hanoi_interrupts, d=diff_hanoi_interrupts))
print('Participants {significance} {change} in the number of interruptions completed during the hanoi task.'.format(
    change=is_improvement(diff_hanoi_interrupts), significance=is_significant(p_hanoi_interrupts)))

# number of math interruptions during hanoi
t_hanoi_math_interrupts, p_hanoi_math_interrupts, diff_hanoi_math_interrupts = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['a_i_name'] == 'math')]['a_i_count'], 
    df_experimental[(df_experimental['te_p_name'] == 'hanoi') & (df_experimental['te_i_name'] == 'math')]['te_i_count'])
print('\nINTERRUPTIONS TOTAL (hanoi-math): t={t}, p={p}, d={d}'.format(t=t_hanoi_math_interrupts, p=p_hanoi_math_interrupts, d=diff_hanoi_math_interrupts))
print('Participants {significance} {change} in the number of math interruptions completed during the hanoi task.'.format(
    change=is_improvement(diff_hanoi_math_interrupts), significance=is_significant(p_hanoi_math_interrupts)))

# number of stroop interruptions during hanoi
t_hanoi_stroop_interrupts, p_hanoi_stroop_interrupts, diff_hanoi_stroop_interrupts = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['a_i_name'] == 'stroop')]['a_i_count'], 
    df_experimental[(df_experimental['te_p_name'] == 'hanoi') & (df_experimental['te_i_name'] == 'stroop')]['te_i_count'])
print('\nINTERRUPTIONS TOTAL (hanoi-stroop): t={t}, p={p}, d={d}'.format(t=t_hanoi_stroop_interrupts, p=p_hanoi_stroop_interrupts, d=diff_hanoi_stroop_interrupts))
print('Participants {significance} {change} in the number of stroop interruptions completed during the hanoi task.'.format(
    change=is_improvement(diff_hanoi_stroop_interrupts), significance=is_significant(p_hanoi_stroop_interrupts)))


INTERRUPTIONS TOTAL (hanoi-any): t=-2.281764241098575, p=0.024128659075414803, d=-6.651515151515151
Participants significantly [*] decreased [-] in the number of interruptions completed during the hanoi task.

INTERRUPTIONS TOTAL (hanoi-math): t=-3.2337178982951857, p=0.002018310830435607, d=-7.7
Participants extremely significantly [**] decreased [-] in the number of math interruptions completed during the hanoi task.

INTERRUPTIONS TOTAL (hanoi-stroop): t=-1.5886050596584365, p=0.11665649866718382, d=-5.777777777777778
Participants  decreased [-] in the number of stroop interruptions completed during the hanoi task.


## Hypotheses
    - training on a novel task (condition = 1)
    - training on a novel interruption (condition = 2)

### Accuracy when training on a novel task (H1)

In [304]:
print('\n-------------- PRIMARY TASKS --------------')

t_1_p_accuracy, p_1_p_accuracy, diff_1_p_accuracy = compute_improvement(
    df_experimental[df_experimental['Condition'] == 1]['a_p_correctness'], 
    df_experimental[df_experimental['Condition'] == 1]['te_p_correctness'])
print('\nACCURACY (any task): t={t}, p={p}, d={d}'.format(t=t_1_p_accuracy, p=p_1_p_accuracy, d=diff_1_p_accuracy))
print('Participants {significance} {change} their accuracy to any primary task when experiencing a novel task.'.format(
    change=is_improvement(diff_1_p_accuracy), significance=is_significant(p_1_p_accuracy)))

t_1_draw_accuracy, p_1_draw_accuracy, diff_1_draw_accuracy = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['Condition'] == 1)]['a_p_correctness'], 
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['Condition'] == 1)]['te_p_correctness'])
print('\nACCURACY (draw): t={t}, p={p}, d={d}'.format(t=t_1_draw_accuracy, p=p_1_draw_accuracy, d=diff_1_draw_accuracy))
print('Participants {significance} {change} their accuracy to draw tasks when experiencing a novel task.'.format(
    change=is_improvement(diff_1_draw_accuracy), significance=is_significant(p_1_draw_accuracy)))

t_1_hanoi_accuracy, p_1_hanoi_accuracy, diff_1_hanoi_accuracy = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['Condition'] == 1)]['a_p_movestotal'].astype(float), 
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['Condition'] == 1)]['te_p_movestotal'].astype(float))
print('\nACCURACY (hanoi): t={t}, p={p}, d={d}'.format(t=t_1_hanoi_accuracy, p=p_1_hanoi_accuracy, d=diff_1_hanoi_accuracy))
print('Participants {significance} {change} their accuracy to hanoi tasks when experiencing a novel task.'.format(
    change=is_improvement(diff_1_hanoi_accuracy), significance=is_significant(p_1_hanoi_accuracy)))

print('\n-------------- INTERRUPTING TASKS --------------')
t_1_i_accuracy, p_1_i_accuracy, diff_1_i_accuracy = compute_improvement(
    df_experimental[df_experimental['Condition'] == 1]['a_i_percentage'], 
    df_experimental[df_experimental['Condition'] == 1]['te_i_percentage'])
print('\nACCURACY (any interruption): t={t}, p={p}, d={d}'.format(t=t_1_i_accuracy, p=p_1_i_accuracy, d=diff_1_i_accuracy))
print('Participants {significance} {change} their accuracy to any interrupting task when experiencing a novel task.'.format(
    change=is_improvement(diff_1_i_accuracy), significance=is_significant(p_1_i_accuracy)))

t_1_stroop_accuracy, p_1_stroop_accuracy, diff_1_stroop_accuracy = compute_improvement(
    df_experimental[(df_experimental['a_i_name'] == 'stroop') & (df_experimental['Condition'] == 1)]['a_i_percentage'], 
    df_experimental[(df_experimental['a_i_name'] == 'stroop') & (df_experimental['Condition'] == 1)]['te_i_percentage'])
print('\nACCURACY (stroop): t={t}, p={p}, d={d}'.format(t=t_1_stroop_accuracy, p=p_1_stroop_accuracy, d=diff_1_stroop_accuracy))
print('Participants {significance} {change} their accuracy to stroop tasks when experiencing a novel task.'.format(
    change=is_improvement(diff_1_stroop_accuracy), significance=is_significant(p_1_stroop_accuracy)))

t_1_math_accuracy, p_1_math_accuracy, diff_1_math_accuracy = compute_improvement(
    df_experimental[(df_experimental['a_i_name'] == 'math') & (df_experimental['Condition'] == 1)]['a_i_percentage'], 
    df_experimental[(df_experimental['a_i_name'] == 'math') & (df_experimental['Condition'] == 1)]['te_i_percentage'])
print('\nACCURACY (math): t={t}, p={p}, d={d}'.format(t=t_1_math_accuracy, p=p_1_math_accuracy, d=diff_1_math_accuracy))
print('Participants {significance} {change} their accuracy to math tasks when experiencing a novel task.'.format(
    change=is_improvement(diff_1_math_accuracy), significance=is_significant(p_1_math_accuracy)))


-------------- PRIMARY TASKS --------------

accuracy (any task): t=-0.5577497899123863, p=0.5780718551803312, d=-0.6875
Participants  decreased [-] their accuracy to any primary task when experiencing a novel task.

accuracy (draw): t=-2.4606837681822857, p=0.016865735716352644, d=-1.375
Participants significantly [*] decreased [-] their accuracy to draw tasks when experiencing a novel task.

accuracy (hanoi): t=4.831259039624397, p=1.0313531913676265e-05, d=6.666666666666667
Participants extremely significantly [**] increased [+] their accuracy to hanoi tasks when experiencing a novel task.

-------------- INTERRUPTING TASKS --------------

accuracy (any interruption): t=-0.36344523622545444, p=0.7169226233183985, d=-0.008620976349999997
Participants  decreased [-] their accuracy to any interrupting task when experiencing a novel task.

accuracy (stroop): t=-0.7787795321459962, p=0.43927546079048363, d=-0.023238403766666672
Participants  decreased [-] their accuracy to stroop tasks 

### Speed when training on a novel task (H1)

In [306]:
print('\n-------------- PRIMARY TASKS --------------')

t_1_p_speed, p_1_p_speed, diff_1_p_speed = compute_improvement(
    df_experimental[df_experimental['Condition'] == 1]['a_p_time'], 
    df_experimental[df_experimental['Condition'] == 1]['te_p_time'])
print('\nTIME SPENT (any task): t={t}, p={p}, d={d}'.format(t=t_1_p_speed, p=p_1_p_speed, d=diff_1_p_speed))
print('Participants {significance} {change} their time spent on any primary task when experiencing a novel task.'.format(
    change=is_improvement(diff_1_p_speed), significance=is_significant(p_1_p_speed)))

t_1_draw_speed, p_1_draw_speed, diff_1_draw_speed = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['Condition'] == 1)]['a_p_time'], 
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['Condition'] == 1)]['te_p_time'])
print('\nTIME SPENT (draw): t={t}, p={p}, d={d}'.format(t=t_1_draw_speed, p=p_1_draw_speed, d=diff_1_draw_speed))
print('Participants {significance} {change} their time spent on draw tasks when experiencing a novel task.'.format(
    change=is_improvement(diff_1_draw_speed), significance=is_significant(p_1_draw_speed)))

t_1_hanoi_speed, p_1_hanoi_speed, diff_1_hanoi_speed = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['Condition'] == 1)]['a_p_time'].astype(float), 
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['Condition'] == 1)]['te_p_time'].astype(float))
print('\nTIME SPENT (hanoi): t={t}, p={p}, d={d}'.format(t=t_1_hanoi_speed, p=p_1_hanoi_speed, d=diff_1_hanoi_speed))
print('Participants {significance} {change} their time spent on hanoi tasks when experiencing a novel task.'.format(
    change=is_improvement(diff_1_hanoi_speed), significance=is_significant(p_1_hanoi_speed)))

print('\n-------------- INTERRUPTING TASKS --------------')
t_1_i_speed, p_1_i_speed, diff_1_i_speed = compute_improvement(
    df_experimental[df_experimental['Condition'] == 1]['a_i_time'], 
    df_experimental[df_experimental['Condition'] == 1]['te_i_time'])
print('\nTIME SPENT (any interruption): t={t}, p={p}, d={d}'.format(t=t_1_i_speed, p=p_1_i_speed, d=diff_1_i_speed))
print('Participants {significance} {change} their time spent on any interrupting task when experiencing a novel task.'.format(
    change=is_improvement(diff_1_i_speed), significance=is_significant(p_1_i_speed)))

t_1_stroop_speed, p_1_stroop_speed, diff_1_stroop_speed = compute_improvement(
    df_experimental[(df_experimental['a_i_name'] == 'stroop') & (df_experimental['Condition'] == 1)]['a_i_time'], 
    df_experimental[(df_experimental['a_i_name'] == 'stroop') & (df_experimental['Condition'] == 1)]['te_i_time'])
print('\nTIME SPENT (stroop): t={t}, p={p}, d={d}'.format(t=t_1_stroop_speed, p=p_1_stroop_speed, d=diff_1_stroop_speed))
print('Participants {significance} {change} their time spent on stroop tasks when experiencing a novel task.'.format(
    change=is_improvement(diff_1_stroop_speed), significance=is_significant(p_1_stroop_speed)))

t_1_math_speed, p_1_math_speed, diff_1_math_speed = compute_improvement(
    df_experimental[(df_experimental['a_i_name'] == 'math') & (df_experimental['Condition'] == 1)]['a_i_time'], 
    df_experimental[(df_experimental['a_i_name'] == 'math') & (df_experimental['Condition'] == 1)]['te_i_time'])
print('\nTIME SPENT (math): t={t}, p={p}, d={d}'.format(t=t_1_math_speed, p=p_1_math_speed, d=diff_1_math_speed))
print('Participants {significance} {change} their time spent on math tasks when experiencing a novel task.'.format(
    change=is_improvement(diff_1_math_speed), significance=is_significant(p_1_math_speed)))


-------------- PRIMARY TASKS --------------

TIME SPENT (any task): t=-0.46106229246582864, p=0.6456026098418916, d=-15.335940557283328
Participants  decreased [-] their time spent on any primary task when experiencing a novel task.

TIME SPENT (draw): t=1.246611190230453, p=0.2175512913090282, d=0.8842440354333334
Participants  increased [+] their time spent on draw tasks when experiencing a novel task.

TIME SPENT (hanoi): t=-1.0546828016141816, p=0.29594416445440286, d=-31.556125149999996
Participants  decreased [-] their time spent on hanoi tasks when experiencing a novel task.

-------------- INTERRUPTING TASKS --------------

TIME SPENT (any interruption): t=3.300562239401442, p=0.001276584850755555, d=0.5711673156333331
Participants extremely significantly [**] increased [+] their time spent on any interrupting task when experiencing a novel task.

TIME SPENT (stroop): t=1.9311870212040623, p=0.05835186563267865, d=0.32739281459999997
Participants  increased [+] their time spen

### Resumption lag when training on a novel task (H1)

In [316]:
t_1_p_resumption, p_1_p_resumption, diff_1_p_resumption = compute_improvement(
    df_experimental[df_experimental['Condition'] == 1]['a_p_resumption'], 
    df_experimental[df_experimental['Condition'] == 1]['te_p_resumption'])
print('\nRESUMPTION LAG (any task): t={t}, p={p}, d={d}'.format(t=t_1_p_resumption, p=p_1_p_resumption, d=diff_1_p_resumption))
print('Participants {significance} {change} their resumption lag on any primary task when experiencing a novel task.'.format(
    change=is_improvement(diff_1_p_resumption), significance=is_significant(p_1_p_resumption)))

t_1_draw_resumption, p_1_draw_resumption, diff_1_draw_resumption = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['Condition'] == 1)]['a_p_resumption'], 
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['Condition'] == 1)]['te_p_resumption'])
print('\nRESUMPTION LAG (draw): t={t}, p={p}, d={d}'.format(t=t_1_draw_resumption, p=p_1_draw_resumption, d=diff_1_draw_resumption))
print('Participants {significance} {change} their resumption lag on draw tasks when experiencing a novel task.'.format(
    change=is_improvement(diff_1_draw_resumption), significance=is_significant(p_1_draw_resumption)))

t_1_hanoi_resumption, p_1_hanoi_resumption, diff_1_hanoi_resumption = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['Condition'] == 1)]['a_p_resumption'].astype(float), 
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['Condition'] == 1)]['te_p_resumption'].astype(float))
print('\nRESUMPTION LAG (hanoi): t={t}, p={p}, d={d}'.format(t=t_1_hanoi_resumption, p=p_1_hanoi_resumption, d=diff_1_hanoi_resumption))
print('Participants {significance} {change} their resumption lag on hanoi tasks when experiencing a novel task.'.format(
    change=is_improvement(diff_1_hanoi_resumption), significance=is_significant(p_1_hanoi_resumption)))


RESUMPTION LAG (any task): t=2.2930244403863895, p=0.023617403154520048, d=1.845040583883334
Participants significantly [*] increased [+] their resumption lag on any primary task when experiencing a novel task.

RESUMPTION LAG (draw): t=1.9964666632162804, p=0.05058427603385491, d=1.6727526028666664
Participants  increased [+] their resumption lag on draw tasks when experiencing a novel task.

RESUMPTION LAG (hanoi): t=7.2267123856839754, p=1.2192483649980985e-09, d=2.0173285648999997
Participants extremely significantly [**] increased [+] their resumption lag on hanoi tasks when experiencing a novel task.


### Interruption lag when training on a novel task (H1)

In [309]:
t_1_p_interruption, p_1_p_interruption, diff_1_p_interruption = compute_improvement(
    df_experimental[df_experimental['Condition'] == 1]['a_p_moveinterruptedtime'], 
    df_experimental[df_experimental['Condition'] == 1]['te_p_moveinterruptedtime'])
print('\nINTERRUPTION LAG (any task): t={t}, p={p}, d={d}'.format(t=t_1_p_interruption, p=p_1_p_interruption, d=diff_1_p_interruption))
print('Participants {significance} {change} their interruption lag on any primary task when experiencing a novel task.'.format(
    change=is_improvement(diff_1_p_interruption), significance=is_significant(p_1_p_interruption)))

t_1_draw_interruption, p_1_draw_interruption, diff_1_draw_interruption = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['Condition'] == 1)]['a_p_moveinterruptedtime'], 
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['Condition'] == 1)]['te_p_moveinterruptedtime'])
print('\nINTERRUPTION LAG (draw): t={t}, p={p}, d={d}'.format(t=t_1_draw_interruption, p=p_1_draw_interruption, d=diff_1_draw_interruption))
print('Participants {significance} {change} their interruption lag on draw tasks when experiencing a novel task.'.format(
    change=is_improvement(diff_1_draw_interruption), significance=is_significant(p_1_draw_interruption)))

t_1_hanoi_interruption, p_1_hanoi_interruption, diff_1_hanoi_interruption = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['Condition'] == 1)]['a_p_moveinterruptedtime'].astype(float), 
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['Condition'] == 1)]['te_p_moveinterruptedtime'].astype(float))
print('\nINTERRUPTION LAG (hanoi): t={t}, p={p}, d={d}'.format(t=t_1_hanoi_interruption, p=p_1_hanoi_interruption, d=diff_1_hanoi_interruption))
print('Participants {significance} {change} their interruption lag on hanoi tasks when experiencing a novel task.'.format(
    change=is_improvement(diff_1_hanoi_interruption), significance=is_significant(p_1_hanoi_interruption)))


INTERRUPTION LAG (any task): t=0.9101713617658861, p=0.3645875823917081, d=0.8708594790000003
Participants  increased [+] their interruption lag on any primary task when experiencing a novel task.

INTERRUPTION LAG (draw): t=12.456026246881779, p=4.920820959342511e-18, d=8.8668473632
Participants extremely significantly [**] increased [+] their interruption lag on draw tasks when experiencing a novel task.

INTERRUPTION LAG (hanoi): t=-7.162840546707094, p=1.5611825671722767e-09, d=-7.125128405200002
Participants extremely significantly [**] decreased [-] their interruption lag on hanoi tasks when experiencing a novel task.


### Accuracy when training on a novel interruption (H2)

In [312]:
print('\n-------------- PRIMARY TASKS --------------')

t_2_p_accuracy, p_2_p_accuracy, diff_2_p_accuracy = compute_improvement(
    df_experimental[df_experimental['Condition'] == 2]['a_p_correctness'], 
    df_experimental[df_experimental['Condition'] == 2]['te_p_correctness'])
print('\nACCURACY (any task): t={t}, p={p}, d={d}'.format(t=t_2_p_accuracy, p=p_2_p_accuracy, d=diff_2_p_accuracy))
print('Participants {significance} {change} their accuracy to any primary task when experiencing a novel interruption.'.format(
    change=is_improvement(diff_2_p_accuracy), significance=is_significant(p_2_p_accuracy)))

t_2_draw_accuracy, p_2_draw_accuracy, diff_2_draw_accuracy = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['Condition'] == 2)]['a_p_correctness'], 
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['Condition'] == 2)]['te_p_correctness'])
print('\nACCURACY (draw): t={t}, p={p}, d={d}'.format(t=t_2_draw_accuracy, p=p_2_draw_accuracy, d=diff_2_draw_accuracy))
print('Participants {significance} {change} their accuracy to draw tasks when experiencing a novel interruption.'.format(
    change=is_improvement(diff_2_draw_accuracy), significance=is_significant(p_2_draw_accuracy)))

t_2_hanoi_accuracy, p_2_hanoi_accuracy, diff_2_hanoi_accuracy = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['Condition'] == 2)]['a_p_movestotal'].astype(float), 
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['Condition'] == 2)]['te_p_movestotal'].astype(float))
print('\nACCURACY (hanoi): t={t}, p={p}, d={d}'.format(t=t_2_hanoi_accuracy, p=p_2_hanoi_accuracy, d=diff_2_hanoi_accuracy))
print('Participants {significance} {change} their accuracy to hanoi tasks when experiencing a novel interruption.'.format(
    change=is_improvement(diff_2_hanoi_accuracy), significance=is_significant(p_2_hanoi_accuracy)))

print('\n-------------- INTERRUPTING TASKS --------------')
t_2_i_accuracy, p_2_i_accuracy, diff_2_i_accuracy = compute_improvement(
    df_experimental[df_experimental['Condition'] == 2]['a_i_percentage'], 
    df_experimental[df_experimental['Condition'] == 2]['te_i_percentage'])
print('\nACCURACY (any interruption): t={t}, p={p}, d={d}'.format(t=t_2_i_accuracy, p=p_2_i_accuracy, d=diff_2_i_accuracy))
print('Participants {significance} {change} their accuracy to any interrupting task when experiencing a novel interruption.'.format(
    change=is_improvement(diff_2_i_accuracy), significance=is_significant(p_2_i_accuracy)))

t_2_stroop_accuracy, p_2_stroop_accuracy, diff_2_stroop_accuracy = compute_improvement(
    df_experimental[(df_experimental['a_i_name'] == 'stroop') & (df_experimental['Condition'] == 2)]['a_i_percentage'], 
    df_experimental[(df_experimental['a_i_name'] == 'stroop') & (df_experimental['Condition'] == 2)]['te_i_percentage'])
print('\nACCURACY (stroop): t={t}, p={p}, d={d}'.format(t=t_2_stroop_accuracy, p=p_2_stroop_accuracy, d=diff_2_stroop_accuracy))
print('Participants {significance} {change} their accuracy to stroop tasks when experiencing a novel interruption.'.format(
    change=is_improvement(diff_2_stroop_accuracy), significance=is_significant(p_2_stroop_accuracy)))

t_2_math_accuracy, p_2_math_accuracy, diff_2_math_accuracy = compute_improvement(
    df_experimental[(df_experimental['a_i_name'] == 'math') & (df_experimental['Condition'] == 2)]['a_i_percentage'], 
    df_experimental[(df_experimental['a_i_name'] == 'math') & (df_experimental['Condition'] == 2)]['te_i_percentage'])
print('\nACCURACY (math): t={t}, p={p}, d={d}'.format(t=t_2_math_accuracy, p=p_2_math_accuracy, d=diff_2_math_accuracy))
print('Participants {significance} {change} their accuracy to math tasks when experiencing a novel interruption.'.format(
    change=is_improvement(diff_2_math_accuracy), significance=is_significant(p_2_math_accuracy)))


-------------- PRIMARY TASKS --------------

ACCURACY (any task): t=-0.2706309263359828, p=0.7870976132110894, d=-0.30970149253731344
Participants  decreased [-] their accuracy to any primary task when experiencing a novel task.

ACCURACY (draw): t=-1.0444204546317966, p=0.30048062300931905, d=-0.6693548387096774
Participants  decreased [-] their accuracy to draw tasks when experiencing a novel task.

ACCURACY (hanoi): t=3.941534602302787, p=0.00018949238160283379, d=4.559027777777778
Participants extremely significantly [**] increased [+] their accuracy to hanoi tasks when experiencing a novel task.

-------------- INTERRUPTING TASKS --------------

ACCURACY (any interruption): t=1.2730003338696176, p=0.20525537576945876, d=0.023388905731343255
Participants  increased [+] their accuracy to any interrupting task when experiencing a novel task.

ACCURACY (stroop): t=0.30350928758264595, p=0.7624020433952218, d=0.004135658499999976
Participants  increased [+] their accuracy to stroop ta

### Speed when training on a novel interruption (H2)

In [317]:
print('\n-------------- PRIMARY TASKS --------------')

t_2_p_speed, p_2_p_speed, diff_2_p_speed = compute_improvement(
    df_experimental[df_experimental['Condition'] == 2]['a_p_time'], 
    df_experimental[df_experimental['Condition'] == 2]['te_p_time'])
print('\nTIME SPENT (any task): t={t}, p={p}, d={d}'.format(t=t_2_p_speed, p=p_2_p_speed, d=diff_2_p_speed))
print('Participants {significance} {change} their time spent on any primary task when experiencing a novel interruption.'.format(
    change=is_improvement(diff_2_p_speed), significance=is_significant(p_2_p_speed)))

t_2_draw_speed, p_2_draw_speed, diff_2_draw_speed = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['Condition'] == 2)]['a_p_time'], 
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['Condition'] == 2)]['te_p_time'])
print('\nTIME SPENT (draw): t={t}, p={p}, d={d}'.format(t=t_2_draw_speed, p=p_2_draw_speed, d=diff_2_draw_speed))
print('Participants {significance} {change} their time spent on draw tasks when experiencing a novel interruption.'.format(
    change=is_improvement(diff_2_draw_speed), significance=is_significant(p_2_draw_speed)))

t_2_hanoi_speed, p_2_hanoi_speed, diff_2_hanoi_speed = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['Condition'] == 2)]['a_p_time'].astype(float), 
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['Condition'] == 2)]['te_p_time'].astype(float))
print('\nTIME SPENT (hanoi): t={t}, p={p}, d={d}'.format(t=t_2_hanoi_speed, p=p_2_hanoi_speed, d=diff_2_hanoi_speed))
print('Participants {significance} {change} their time spent on hanoi tasks when experiencing a novel interruption.'.format(
    change=is_improvement(diff_2_hanoi_speed), significance=is_significant(p_2_hanoi_speed)))

print('\n-------------- INTERRUPTING TASKS --------------')
t_2_i_speed, p_2_i_speed, diff_2_i_speed = compute_improvement(
    df_experimental[df_experimental['Condition'] == 2]['a_i_time'], 
    df_experimental[df_experimental['Condition'] == 2]['te_i_time'])
print('\nTIME SPENT (any interruption): t={t}, p={p}, d={d}'.format(t=t_2_i_speed, p=p_2_i_speed, d=diff_2_i_speed))
print('Participants {significance} {change} their time spent on any interrupting task when experiencing a novel interruption.'.format(
    change=is_improvement(diff_2_i_speed), significance=is_significant(p_2_i_speed)))

t_2_stroop_speed, p_2_stroop_speed, diff_2_stroop_speed = compute_improvement(
    df_experimental[(df_experimental['a_i_name'] == 'stroop') & (df_experimental['Condition'] == 2)]['a_i_time'], 
    df_experimental[(df_experimental['a_i_name'] == 'stroop') & (df_experimental['Condition'] == 2)]['te_i_time'])
print('\nTIME SPENT (stroop): t={t}, p={p}, d={d}'.format(t=t_2_stroop_speed, p=p_2_stroop_speed, d=diff_2_stroop_speed))
print('Participants {significance} {change} their time spent on stroop tasks when experiencing a novel interruption.'.format(
    change=is_improvement(diff_2_stroop_speed), significance=is_significant(p_2_stroop_speed)))

t_2_math_speed, p_2_math_speed, diff_2_math_speed = compute_improvement(
    df_experimental[(df_experimental['a_i_name'] == 'math') & (df_experimental['Condition'] == 2)]['a_i_time'], 
    df_experimental[(df_experimental['a_i_name'] == 'math') & (df_experimental['Condition'] == 2)]['te_i_time'])
print('\nTIME SPENT (math): t={t}, p={p}, d={d}'.format(t=t_2_math_speed, p=p_2_math_speed, d=diff_2_math_speed))
print('Participants {significance} {change} their time spent on math tasks when experiencing a novel interruption.'.format(
    change=is_improvement(diff_2_math_speed), significance=is_significant(p_2_math_speed)))


-------------- PRIMARY TASKS --------------

TIME SPENT (any task): t=-0.6070174818941079, p=0.5448820167517792, d=-20.35177128347762
Participants  decreased [-] their time spent on any primary task when experiencing a novel interruption.

TIME SPENT (draw): t=1.7309862302473986, p=0.08859416555004096, d=1.1235444873225806
Participants  increased [+] their time spent on draw tasks when experiencing a novel interruption.

TIME SPENT (hanoi): t=-1.1491282767255362, p=0.2544144149258241, d=-38.84440430833334
Participants  decreased [-] their time spent on hanoi tasks when experiencing a novel interruption.

-------------- INTERRUPTING TASKS --------------

TIME SPENT (any interruption): t=2.609236678985934, p=0.010121612898473121, d=0.4045483994328359
Participants significantly [*] increased [+] their time spent on any interrupting task when experiencing a novel interruption.

TIME SPENT (stroop): t=2.8758232228003062, p=0.005335551874903006, d=0.2854687288611111
Participants significant

### Resumption lag when training on a novel interruption (H2)

In [318]:
t_2_p_resumption, p_2_p_resumption, diff_2_p_resumption = compute_improvement(
    df_experimental[df_experimental['Condition'] == 2]['a_p_resumption'], 
    df_experimental[df_experimental['Condition'] == 2]['te_p_resumption'])
print('\nRESUMPTION LAG (any task): t={t}, p={p}, d={d}'.format(t=t_2_p_resumption, p=p_2_p_resumption, d=diff_2_p_resumption))
print('Participants {significance} {change} their resumption lag on any primary task when experiencing a novel interruption.'.format(
    change=is_improvement(diff_2_p_resumption), significance=is_significant(p_2_p_resumption)))

t_2_draw_resumption, p_2_draw_resumption, diff_2_draw_resumption = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['Condition'] == 2)]['a_p_resumption'], 
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['Condition'] == 2)]['te_p_resumption'])
print('\nRESUMPTION LAG (draw): t={t}, p={p}, d={d}'.format(t=t_2_draw_resumption, p=p_2_draw_resumption, d=diff_2_draw_resumption))
print('Participants {significance} {change} their resumption lag on draw tasks when experiencing a novel interruption.'.format(
    change=is_improvement(diff_2_draw_resumption), significance=is_significant(p_2_draw_resumption)))

t_2_hanoi_resumption, p_2_hanoi_resumption, diff_2_hanoi_resumption = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['Condition'] == 2)]['a_p_resumption'].astype(float), 
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['Condition'] == 2)]['te_p_resumption'].astype(float))
print('\nRESUMPTION LAG (hanoi): t={t}, p={p}, d={d}'.format(t=t_2_hanoi_resumption, p=p_2_hanoi_resumption, d=diff_2_hanoi_resumption))
print('Participants {significance} {change} their resumption lag on hanoi tasks when experiencing a novel interruption.'.format(
    change=is_improvement(diff_2_hanoi_resumption), significance=is_significant(p_2_hanoi_resumption)))


RESUMPTION LAG (any task): t=3.458550046493689, p=0.0007315639618033416, d=2.3348994205671647
Participants extremely significantly [**] increased [+] their resumption lag on any primary task when experiencing a novel interruption.

RESUMPTION LAG (draw): t=2.982893010624936, p=0.004123534626083591, d=2.0889182728064517
Participants extremely significantly [**] increased [+] their resumption lag on draw tasks when experiencing a novel interruption.

RESUMPTION LAG (hanoi): t=8.218261495352612, p=7.238731309800765e-12, d=2.5467165200277777
Participants extremely significantly [**] increased [+] their resumption lag on hanoi tasks when experiencing a novel interruption.


### Interruption lag when training on a novel interruption (H2)

In [315]:
t_2_p_interruption, p_2_p_interruption, diff_2_p_interruption = compute_improvement(
    df_experimental[df_experimental['Condition'] == 2]['a_p_moveinterruptedtime'], 
    df_experimental[df_experimental['Condition'] == 2]['te_p_moveinterruptedtime'])
print('\nINTERRUPTION LAG (any task): t={t}, p={p}, d={d}'.format(t=t_2_p_interruption, p=p_2_p_interruption, d=diff_2_p_interruption))
print('Participants {significance} {change} their interruption lag on any primary task when experiencing a novel interruption.'.format(
    change=is_improvement(diff_2_p_interruption), significance=is_significant(p_2_p_interruption)))

t_2_draw_interruption, p_2_draw_interruption, diff_2_draw_interruption = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['Condition'] == 2)]['a_p_moveinterruptedtime'], 
    df_experimental[(df_experimental['a_p_name'] == 'draw') & (df_experimental['Condition'] == 2)]['te_p_moveinterruptedtime'])
print('\nINTERRUPTION LAG (draw): t={t}, p={p}, d={d}'.format(t=t_2_draw_interruption, p=p_2_draw_interruption, d=diff_2_draw_interruption))
print('Participants {significance} {change} their interruption lag on draw tasks when experiencing a novel interruption.'.format(
    change=is_improvement(diff_2_draw_interruption), significance=is_significant(p_2_draw_interruption)))

t_2_hanoi_interruption, p_2_hanoi_interruption, diff_2_hanoi_interruption = compute_improvement(
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['Condition'] == 2)]['a_p_moveinterruptedtime'].astype(float), 
    df_experimental[(df_experimental['a_p_name'] == 'hanoi') & (df_experimental['Condition'] == 2)]['te_p_moveinterruptedtime'].astype(float))
print('\nINTERRUPTION LAG (hanoi): t={t}, p={p}, d={d}'.format(t=t_2_hanoi_interruption, p=p_2_hanoi_interruption, d=diff_2_hanoi_interruption))
print('Participants {significance} {change} their interruption lag on hanoi tasks when experiencing a novel interruption.'.format(
    change=is_improvement(diff_2_hanoi_interruption), significance=is_significant(p_2_hanoi_interruption)))


INTERRUPTION LAG (any task): t=3.4805980698582997, p=0.0006785948905805176, d=2.3277089995522386
Participants extremely significantly [**] increased [+] their interruption lag on any primary task when experiencing a novel task.

INTERRUPTION LAG (draw): t=3.111220493472856, p=0.0028511983446629597, d=2.110305370548387
Participants extremely significantly [**] increased [+] their interruption lag on draw tasks when experiencing a novel task.

INTERRUPTION LAG (hanoi): t=8.032329582869417, p=1.5927570945094394e-11, d=2.5149176800833337
Participants extremely significantly [**] increased [+] their interruption lag on hanoi tasks when experiencing a novel task.
